# Harmonization across studies with PIC-SURE

This tutorial notebook will demonstrate how to query and work with the BioData Catalyst studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

-----

# Environment set-up

### System Requirements
R >= 3.4

### Install packages

**Note that if you are using the dedicated PIC-SURE environment within the BioData Catalyst Seven Bridges platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in Seven Bridges, or if you do not have all the necessary dependencies installed.
#install.packages(c("devtools", "dplyr", "ggplot2"))

In [ ]:
install.packages('devtools')
devtools::install_github("hms-dbmi/pic-sure-r-client", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", ref="new-search", force=T)
library(dplyr)
library(ggplot2)
library(hpds)

In [ ]:
install.packages("BiocManager", verbose=TRUE)
library(BiocManager)
BiocManager::install("limma")
library(limma)

## Connecting to a PIC-SURE Network

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
connection <- picsure::connect(PICSURE_network_URL, token)
authPicSure = bdc::use.authPicSure(connection)

-----

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate an approach to finding and extracting similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the *"orthopnea"* and *"pneumonia"* variables across studies


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the [`2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb`](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/blob/master/NHLBI_BioData_Catalyst/python/2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb) notebook*

-----

## Sex and gender variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for sex and BMI which combine data from multiple studies**</font> 

These variables are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

Let's start by searching for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms. The `bdc::find.in.dictionary()` function can take in regular expressions. Here, we take advantage of this by searching for both sex and gender in the same regular expression.


In [ ]:
dictionary <- bdc::use.dictionary(connection) # set up the variable dictionary
sex_dictionary <- bdc::find.in.dictionary(dictionary, "sex|gender")
sex_df <- bdc::extract.dataframe(sex_dictionary)
head(sex_df, 3)

After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) (phs001252)
- EOCOPD (Early Onset of COPD) (phs000946)

We can find the study IDs in the Data Access Dashboard in the user interface.

We have already used the PIC-SURE API dictionary object and find method to find variables which contain the keywords sex and gender. Let's filter these results to our desired studies of interest.

In [ ]:
# which sex / gender variables are part of ECLIPSE?
# create a subset of only ECLIPSE sex/gender vars
eclipse_sex_df <- sex_df %>%
    filter(grepl('phs001252', study_id)) %>%
    select(var_name, var_description, values, HPDS_PATH)

eclipse_sex_df

Many studies may have multiple variables with similar names and descriptions that come from different data tables. This will vary from study to study based on how the original study was conducted and organized. 

We will approach this complication by examining the data associated with each sex / gender variables in ECLIPSE and determining which would be the best fit for our analysis.

Because we want to examine the patient level data, we will create a query using all sex/gender variables in ECLIPSE. We will add these variables to our query using the `anyof` method, as we are interested in all observations with a value for any of our chosen variables.

In [ ]:
eclipse_sex_query <- bdc::new.query(authPicSure) # Start a new query
invisible(lapply(eclipse_sex_df$HPDS_PATH, bdc::query.anyof.add, query = eclipse_sex_query))
eclipse_sex_results <- bdc::query.run(eclipse_sex_query)
head(eclipse_sex_results)

By previewing the resulting dataframe, we can see that not all of the sex and gender variables are complete for our participant subset. 

Let's see which sex/gender variables are the most complete for our dataset by counting the number of empty values in each one. 

In [ ]:
# first let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values
complete_df <- as.data.frame(colSums(eclipse_sex_results == ''))
colnames(complete_df) <- 'n_empty'
complete_df


We can see that there are a few variables with 0 NA values, we would like to focus on these.

In [ ]:
filtered_eclipse_sex_results <- eclipse_sex_results %>%
    select(complete_df %>% filter(n_empty == 0) %>% rownames)
head(filtered_eclipse_sex_results)

Now we have filtered our dataset to only include sex/gender variables in ECLIPSE which have complete data. Let's check to see if the values in each variable (column) differ, or if they all have the same data.

In [ ]:
# in this example, we notice that all our filtered variables have equivalent data for each patient:
unique(apply(filtered_eclipse_sex_results[,-c(1:3)], 1, n_distinct))

#filtered_eclipse_sex_results.iloc[:,4:].nunique(axis = 1).unique()

We can see that all the values in the filtered, complete ECLIPSE sex/gender variables are the same, so we will simply select the first variable to use in our analysis.

We can use the `get.varInfo()` function to learn more about a variable.

In [ ]:
eclipse_sex_var = colnames(filtered_eclipse_sex_results)[4]
bdc::get.varInfo(sex_dictionary, eclipse_sex_var)

### Which sex variable in EOCOPD should we use? 

We will take a similar approach as above, where we first filter our dataframe of sex/gender variables to only those from the EOCOPD study.

In [ ]:
# which sex/gender variables are part of EOCOPD?
eocopd_vars <- sex_df %>% 
    filter(grepl('phs000946', study_id))

Unlike the ECLIPSE data, we see that there is only one sex variable in EOCOPD. It is called 'GENDER'.


In [ ]:
# create a subset of only EOCOPD sex/gender vars
eocopd_sex_df <- eocopd_vars %>%
    select(var_name, var_description, values, HPDS_PATH)
eocopd_sex_df

We will save the HPDS_PATH associated to this variable so we can add it to our query later.

In [ ]:
# save the HPDS path associated to our variable of interest
eocopd_sex_var = eocopd_sex_df %>% pull(HPDS_PATH)

### Which BMI variables should we use?

We will follow the approach outlined above to first search our data dictionary for all variables containing the terms 'body mass index'. 

In [ ]:
bmi_dictionary <- bdc::find.in.dictionary(dictionary, "body mass index")
bmi_df <- bdc::extract.dataframe(bmi_dictionary)


We then filter to only those BMI variables which are part of ECLIPSE or EOCOPD and view the variables with their associated metadata. 

In [ ]:
# which bmi variables are part of ECLIPSE or EOCOPD?
# create a subset of only the ECLIPSE / EOCOPD BMI vars
bmi_df <- bmi_df %>%
    filter(grepl('phs001252|phs000946', study_id)) %>%
    select(var_name, var_description, values, HPDS_PATH)

head(bmi_df)


By examining metadata like the variable name and description, we are able to determine which variables we are interested in:
- Body Mass Index [BMI ] for EOCOPD (\phs000946\pht005719\phv00266123\BMI\)
- Body mass index (kg/m2) from the bodycomp table for ECLIPSE (\phs001252\pht006373\phv00293610\VSBMI\) 

We will save the HPDS_PATHs associated to the BMI variables of interest so we can add it to our query later.

In [ ]:
# save HPDS_PATHs of interest
eocopd_bmi_var <- "\\phs000946\\pht005719\\phv00266123\\BMI\\"
eclipse_bmi_var <- "\\phs001252\\pht006373\\phv00293610\\VSBMI\\"


### Query PIC-SURE for selected variables
First, we create a new query:

In [ ]:
combined_query <- bdc::new.query(authPicSure) # Start a new query

Next, we add the HPDS paths (concept paths) for our selected variabels of interest to the query.

As a reminder, these are the paths associated to the sex/gender and BMI variables from ECLIPSE and EOCOPD.

We use the `.anyof` method to add these variables to the query because we are interested in results with observations in any of the variables of interest. If you are interested in learning more about the other query methods available, see the `1_PICSURE_API_101.ipynb` notebook. 

In [ ]:
# Build query using selected concept paths
vars_of_interest <- c(eclipse_sex_var, eclipse_bmi_var, eocopd_sex_var, eocopd_bmi_var)
invisible(lapply(vars_of_interest, bdc::query.anyof.add, query = combined_query))
combined_results <- bdc::query.run(combined_query)
head(combined_results)

### Clean and wrangle query result dataframe
You may notice that the column names are not the most informative, we will rename them below.

In [ ]:
# rename columns
colnames(combined_results) = c('patient_id', 'parent_accession', 'topmed_accession', 'consent', 
                               'eocopd_sex', 'eocopd_bmi', 'eclipse_bmi', 'eclipse_sex')

We will also create a 'study' column to easier identify from which study a participant's data originated from.

In [ ]:
# convert consent code to study name
combined_results <- combined_results %>%
    mutate(study = ifelse(grepl('phs001252', consent), 'eclipse', 'eocopd'))

head(combined_results)


Finally, we will harmonize the sex and bmi variables by combining them into one single 'sex' column.

Sex and BMI are reported in the same way in both EOCOPD and ECLIPSE, so we do not have to transform any of the participant level data.

In [ ]:
combined_results <- combined_results %>%
           # combine sex variables
    mutate(sex = eocopd_sex, 
           sex = ifelse(sex == '', eclipse_sex, sex),
           # combine bmi variables
           bmi = eocopd_bmi,
           bmi = ifelse(is.nan(bmi), eclipse_bmi, bmi))

head(combined_results)


### Analysis and visualization

In this example, we are interested in comparing mean BMI across different studies. We will calculate the mean for each group (ECLIPSE males, ECLIPSE females, EOCOPD males, EOCOPD females) and display the results as a table and a bar chart.

In [ ]:
# remove unneccessary columns and summarize
plotdf = combined_results %>% 
    select(sex, study, bmi) %>%
    group_by(sex, study) %>%
    summarise(mean = mean(bmi, na.rm = TRUE))

plotdf

In [ ]:
plotdf %>%
ggplot(aes(x = sex, y = mean, fill = study)) +
    geom_col(position="dodge") +
    theme_bw()


## Orthopnea variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for orthopnea and pneumonia which combine data from multiple studies, and compare participant distributions across these phenotypes**</font> 

In this example, we will be harmonizing variables related to `orthopnea`. Orthopnea is shortness of breath that occurs when individuals lie flat. Because of this, people with orthopnea have to sleep propped up in bed or sitting in a chair. You can read more about this condition [here](https://www.sleepfoundation.org/sleep-apnea/orthopnea).

Let's start by doing a search in PIC-SURE for concept paths containing `orthopnea`.

### Search and find orthopnea variables


In [ ]:
orthopnea_dictionary <- bdc::find.in.dictionary(dictionary, "orthopnea")
orthopnea_dataframe <- bdc::extract.dataframe(orthopnea_dictionary)
head(orthopnea_dataframe, 3) %>% select(var_name, var_description, values, study_id)

As shown in the resulting dataframe, orthopnea is often recorded as a something like `sleep on two or more pillows to help you breathe?`

Let's do a search for `pillows` in PIC-SURE to explore those concept paths.

In [ ]:
pillows_dictionary <- bdc::find.in.dictionary(dictionary, "pillows")
pillows_dataframe <- bdc::extract.dataframe(pillows_dictionary)
head(pillows_dataframe, 3) %>% select(var_name, var_description, values, study_id)


As shown in the resulting dataframe, there are some concept paths related to orthopnea that were not captured by the previous `orthopnea` search, such as `Do you sleep on 2 or more pillows to improve your breathing?` from the Jackson Heart Study (JHS) Cohort (phs000286). In fact, the JHS dataset was not returned at all when we searched for `orthopnea`. 

This is problematic for researchers studying these types of variables, since the concept paths of interest may not always be captured by a single search term. To account for this, researchers must conduct exploratory searches to determine potential search terms.

Let's say we want to harmonize orthopnea variables from the following datasets: 
- FHS (phs000007)
- MESA (phs000209)
- WHI (phs000200)

We can first get all concept paths related to our terms of interest (`orthopnea` or `pillows`) and then filter to our studies of interest.

You can get the phs number associated with each study from the Data Access Dashboard.

In [ ]:
harmonized_dictionary <- bdc::find.in.dictionary(dictionary, "orthopnea|pillows")
harmonized_df <- bdc::extract.dataframe(harmonized_dictionary)
harmonized_df <- harmonized_df %>%
    filter(grepl('phs000007|phs000209|phs000200', study_id))
harmonized_df %>% select(var_name, var_description, values, study_id)


After browsing the available variables, we decided to choose the following:

In [ ]:
orthopnea_variables_of_interest <- harmonized_df %>%
    filter(var_description %in% c('Orthopnea', 
                                  'F136 Orthopnea', 
                                  'SLEEP ON 2+ PILLOWS TO HELP BREATHING'))

# save variable paths for querying data
orthopnea_variable_paths_of_interest = orthopnea_variables_of_interest$HPDS_PATH

orthopnea_variables_of_interest %>% select(var_name, var_description, values, study_id)

### Search and find pneumonia variables
As part of our research, let's say we are interested in exploring the relationship between pneumonia and orthopnea. Let's save concept paths related to `pneumonia` as well. 

In [ ]:
# search pic-sure for 'pneumonia'
harmonized_dictionary <- bdc::find.in.dictionary(dictionary, "pneumonia")
harmonized_df <- bdc::extract.dataframe(harmonized_dictionary)
harmonized_df <- harmonized_df %>%
    # filter to variables found in FHS, MESA, WHI
    filter(grepl('phs000007|phs000209|phs000200', study_id))
harmonized_df %>% select(var_name, var_description, values, study_id)


After browsing the available variables, we decided to choose the following:

In [ ]:
pneumonia_variables_of_interest = harmonized_df %>%
    filter(grepl('phv00255144|phv00083774|phv00283208', var_id))

# save variable paths for querying data
pneumonia_variable_paths_of_interest = pneumonia_variables_of_interest$HPDS_PATH

pneumonia_variables_of_interest %>% select(var_name, var_description, values, study_id)


### Query PIC-SURE for selected variables

In [ ]:
orthopnea_query <- bdc::new.query(authPicSure) # Start a new query
# Build query using selected concept paths
vars_of_interest <- c(orthopnea_variable_paths_of_interest, pneumonia_variable_paths_of_interest)
invisible(lapply(vars_of_interest, bdc::query.anyof.add, query = orthopnea_query))
orthopnea_results <- bdc::query.run(orthopnea_query)
head(orthopnea_results)


In [ ]:
# rename columns
colnames(orthopnea_results) = c('patient_id', 'parent_accession', 'topmed_accession', 'consent', 
                             'fhs_pneumonia', 'fhs_orthopnea', 'whi_pneumonia', 
                             'whi_orthopnea', 'mesa_pneumonia', 'mesa_orthopnea')
head(orthopnea_results)


### Clean and wrangle data

In [ ]:
orthopnea_results <- orthopnea_results %>%
    # convert consent code to study name
    mutate(study = ifelse(grepl('phs000007', consent), 'fhs',
                         ifelse(grepl('phs000209', consent), 'mesa', 'whi')),
          # combine orthopnea variables
          orthopnea = ifelse(study == 'fhs', fhs_orthopnea,
                             ifelse(study == 'mesa', mesa_orthopnea,
                                    ifelse(study == 'whi', whi_orthopnea,
                                           'OTHER'))),
          # combine orthopnea variables
          pneumonia = ifelse(study == 'fhs', fhs_pneumonia,
                             ifelse(study == 'mesa', mesa_pneumonia,
                                    ifelse(study == 'whi', whi_pneumonia,
                                           'OTHER'))))


head(orthopnea_results)


As you may have noticed, the orthopnea and pneumonia variables are encoded differently between these studies. 

This means that we see a range of vales that need to be harmonized (printed below)

We need to create a consistent encoding across these studies. 

In [ ]:
print('Orthopnea values: ')
table(orthopnea_results$orthopnea)

print('Pneumonia values: ')
table(orthopnea_results$pneumonia)

We create a consistent mapping:

In [ ]:
mapping_df <- data.frame('raw' = c(unique(orthopnea_results$orthopnea), 
                                   unique(orthopnea_results$pneumonia)),
                        'harmonized' = c('Yes', 'No', 'Unknown', 'No', 'Yes', 'Unknown',
                                         'No', 'Unknown', 'No', 'Yes', 'Unknown', 'No',
                                         'Yes', 'Unknown')) %>%
unique()

mapping_df
        

And apply it to our hamonized orthopnea and pneumonia columns

In [ ]:
orthopnea_results <- left_join(orthopnea_results, mapping_df, by = c('orthopnea' = 'raw'))
orthopnea_results$orthopnea <- orthopnea_results$harmonized
orthopnea_results <- left_join(orthopnea_results, mapping_df, by = c('pneumonia' = 'raw'))
orthopnea_results$pneumonia <- orthopnea_results$harmonized.y


table(orthopnea_results$orthopnea)
table(orthopnea_results$pneumonia)


Now that the variables have been encoded in the same way, we can use this dataframe for analysis and visualizations. 

### Visualize

In [ ]:
final_df <- orthopnea_results

In [ ]:
final_df %>% 
    filter(study == 'fhs') %>%
    select(pneumonia, orthopnea) %>%
    filter(pneumonia != 'Unknown',
           orthopnea != 'Unknown') %>%
    mutate(pneumonia = ifelse(pneumonia == 'Yes', 1, 0),
           orthopnea = ifelse(orthopnea == 'Yes', 1, 0)) %>%
    as.matrix %>%
    vennCounts %>%
    vennDiagram(main = 'FHS Dataset')

final_df %>% 
    filter(study == 'mesa') %>%
    select(pneumonia, orthopnea) %>%
    filter(pneumonia != 'Unknown',
           orthopnea != 'Unknown') %>%
    mutate(pneumonia = ifelse(pneumonia == 'Yes', 1, 0),
           orthopnea = ifelse(orthopnea == 'Yes', 1, 0)) %>%
    as.matrix %>%
    vennCounts %>%
    vennDiagram(main = 'MESA Dataset')

final_df %>% 
    filter(study == 'whi') %>%
    select(pneumonia, orthopnea) %>%
    filter(pneumonia != 'Unknown',
           orthopnea != 'Unknown') %>%
    mutate(pneumonia = ifelse(pneumonia == 'Yes', 1, 0),
           orthopnea = ifelse(orthopnea == 'Yes', 1, 0)) %>%
    as.matrix %>%
    vennCounts %>%
    vennDiagram(main = 'WHI Dataset')

final_df %>% 
    select(pneumonia, orthopnea) %>%
    filter(pneumonia != 'Unknown',
           orthopnea != 'Unknown') %>%
    mutate(pneumonia = ifelse(pneumonia == 'Yes', 1, 0),
           orthopnea = ifelse(orthopnea == 'Yes', 1, 0)) %>%
    as.matrix %>%
    vennCounts %>%
    vennDiagram(main = 'Combined results across FHS, MESA, and WHI')
